# Restarting core files models after a core model run error

The simplest way to setup and run experiments, and then to import the results and archive the model is to use `run_experiments`. However, if the core files model fails during an experiment and it is preferable to not restart that particular experiment from scratch but to finish the run manually, the individual functions of the function may be called before running the remaining experiments.

This example first shows the process to run a set of designed experiments through the core model, then shows the steps to run the individual functions of `run_experiments`. Finally, the command to run the remaining functions is shown using the pending_only argument of run_experiments.  

This example assumes that an experiment design has been created, for examples on creating an 
experiment design, see the examples in the `design_experiments`


In [2]:
import emat
emat.__version__

'0.1.4'

In [2]:
import pandas, numpy, os
from emat.util.loggers import log_to_stderr
log = log_to_stderr(level=20)

In [3]:
# connect to database of experiments
db = emat.SQLiteDB("emat_db/gbnrtc.db",initialize=False)
scope = db.read_scope('GBNRTC')

In [4]:
# load model object
from emat.model import GBNRTCModel

In [5]:
g = GBNRTCModel(
    configuration='gbnrtc_model_config.yaml',
    scope=scope,
    db=db,
)
g

## Run All Experiments

If all goes well and the core model will be run on a single system, the following function is the only one necessary to run to execute the core model across the set of experiments and populate the database with the model outputs.

In [6]:
exp_df = g.read_experiments(design_name='lhs')
exp_df.head()

,Land Use - CBD Focus,Freeway Capacity,Auto IVTT Sensitivity,Shared Mobility,Kensington Decommissioning,LRT Extension
experiment,,,,,,
1,1.086300,1.587369,0.873289,0.425711,True,True
2,1.023312,1.158360,0.911751,0.252330,False,True
3,1.170003,1.301395,0.817217,0.509674,False,True
4,1.096142,1.634592,0.858481,0.154432,True,True
5,0.844650,1.348292,0.808333,0.288354,True,True


In [ ]:
g.run_experiments()

## Restart single experiment after manual completion of run

Now let's assume that during the first experiment (ID 1) the model crashed on the last speed-feedback iteration. Rather than restart the model run at the beginning (as run_experiments would do) the user may complete the model run manually and then post process and import the model outputs using TMIP-EMAT

If the user prefers to simply restart the core model from the beginning, skip this step and proceed to the following section on Running Pending Experiments.

The first step is to identify the experiment that failed. This should be displayed in the error message from the command above.

In [7]:
# let's assume that the model failed on the 1st experiment, 
# note this matches the experiment column in the data frame above
exp_id = 1

# get the measure names from the scope for reference
measure_names = g.scope.get_measure_names()

# get a dictionary of the parameters for reference
params = exp_df.loc[exp_id]
params

Land Use - CBD Focus            1.0863
Freeway Capacity               1.58737
Auto IVTT Sensitivity         0.873289
Shared Mobility               0.425711
Kensington Decommissioning        True
LRT Extension                     True
Name: 1, dtype: object

In [8]:
# First, call the post processor to generate the output metrics in a format 
# that tmip-emat can read

g.post_process(params, measure_names)

[02:18.44] MainProcess/INFO: Running post process scripts at 2019_08_07 141145
[02:18.45] MainProcess/WARNING: [(re)starting TransCAD instances]


In [9]:
# Next, import the metrics and write them to the database
measures_dictionary = g.load_measures(measure_names)
m_df = pandas.DataFrame(measures_dictionary, index=[exp_id])

# Assign to outcomes_output, for ema_workbench compatibility
g.outcomes_output = measures_dictionary
g.db.write_experiment_measures(g.scope.name, g.metamodel_id, m_df)

In [11]:
# confirm that the metrics were imported into the database
g.read_experiments(design_name='lhs').head()

,Land Use - CBD Focus,Freeway Capacity,Auto IVTT Sensitivity,Shared Mobility,Kensington Decommissioning,LRT Extension,Region-wide VMT,Interstate + Expressway + Ramp/Connector VMT,Major and Minor Arterials VMT,Total Auto VMT,...,OD Volume District 1 to 7,OD Volume District 1 to 8,OD Volume District 1 to 9,OD Volume District 1 to 10,Peak Transit Share,Peak NonMotorized Share,Off-Peak Transit Share,Off-Peak NonMotorized Share,Daily Transit Share,Daily NonMotorized Share
experiment,,,,,,,,,,,,,,,,,,,,,
1,1.086300,1.587369,0.873289,0.425711,True,True,2.577793e+07,1.064360e+07,1.073000e+07,2.305935e+07,...,22135.892578,1957.864014,10585.642578,4567.508301,0.021358,0.072049,0.017564,0.06862,0.019305,0.070193
2,1.023312,1.158360,0.911751,0.252330,False,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.170003,1.301395,0.817217,0.509674,False,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.096142,1.634592,0.858481,0.154432,True,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.844650,1.348292,0.808333,0.288354,True,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# finally, archive the model
archive_path = g.get_experiment_archive_path(exp_id)
g.archive(params, archive_path, exp_id)

## Running Pending Experiments

Now that the partial run has been wrapped up, imported, and archived, we can resume running the remaining experiments, with fingers crossed!

To identify the experiments that haven't been run, we can identify the pending experiments and pass those to the run_experiments method.

In [15]:
# reading in the pending experiments, note that experiment 1 is not included
pending_exp = g.read_experiments(design_name='lhs',only_pending=True)
pending_exp.head()

,Land Use - CBD Focus,Freeway Capacity,Auto IVTT Sensitivity,Shared Mobility,Kensington Decommissioning,LRT Extension,Region-wide VMT,Interstate + Expressway + Ramp/Connector VMT,Major and Minor Arterials VMT,Total Auto VMT,...,OD Volume District 1 to 7,OD Volume District 1 to 8,OD Volume District 1 to 9,OD Volume District 1 to 10,Peak Transit Share,Peak NonMotorized Share,Off-Peak Transit Share,Off-Peak NonMotorized Share,Daily Transit Share,Daily NonMotorized Share
experiment,,,,,,,,,,,,,,,,,,,,,
2,1.023312,1.158360,0.911751,0.252330,False,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.170003,1.301395,0.817217,0.509674,False,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.096142,1.634592,0.858481,0.154432,True,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.844650,1.348292,0.808333,0.288354,True,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.883839,1.492314,0.944300,0.651629,True,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
g.run_experiments(design=pending_exp)